In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,split
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
spark = (SparkSession
 .builder
 .appName("Python")
 .getOrCreate())

22/08/10 12:33:33 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/08/10 12:33:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/10 12:33:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

sc = StructType([
        StructField('analytics', StructType([
             StructField('clicks', IntegerType(), True),
             StructField('impressions', IntegerType(), True)
             ])),
         StructField('datetime', TimestampType(), True),
         StructField('sales', StructType([
             StructField('quantity', IntegerType(), True),
             StructField('total_price', DoubleType(), True)
             ]))
         ])

print(sc)



StructType(List(StructField(analytics,StructType(List(StructField(clicks,IntegerType,true),StructField(impressions,IntegerType,true))),true),StructField(datetime,TimestampType,true),StructField(sales,StructType(List(StructField(quantity,IntegerType,true),StructField(total_price,DoubleType,true))),true)))


In [44]:
df = spark.read.format('json').schema(sc).load('/home/ubuntu/Spark/Spark_Work/python_file/data/json_files/file_0.json')

In [79]:
df.show()

#a = df.select("analytics.clicks")

df = df.withColumn('clicks',df.analytics.clicks)
#df.show()

df = df.withColumn('impressions',df.analytics.impressions)

df = df.withColumn('quantity',df.sales.quantity)
df = df.withColumn('total_price',df.sales.total_price)

df = df.drop("analytics","sales")

df.show()

#a =  df.withColumn("Click",col("analytics").cast('string'))

df.schema

+---------+--------------------+--------------------+------+-----------+--------+-----------------+
|analytics|            datetime|               sales|clicks|impressions|quantity|      total_price|
+---------+--------------------+--------------------+------+-----------+--------+-----------------+
|  {6, 13}|2022-08-07T04:21:...|            {5, 6.4}|     6|         13|       5|              6.4|
|  {4, 11}|2022-08-07T04:21:...|            {6, 6.5}|     4|         11|       6|              6.5|
|  {7, 19}|2022-08-07T04:21:...|            {9, 6.5}|     7|         19|       9|              6.5|
|  {4, 19}|2022-08-07T04:21:...|           {10, 5.1}|     4|         19|      10|              5.1|
|  {5, 18}|2022-08-07T04:21:...|            {3, 0.8}|     5|         18|       3|              0.8|
|  {9, 16}|2022-08-07T04:21:...|{7, 9.20000000000...|     9|         16|       7|9.200000000000001|
|  {8, 12}|2022-08-07T04:21:...|            {2, 6.3}|     8|         12|       2|              6.3|


StructType(List(StructField(datetime,StringType,true),StructField(clicks,StringType,true),StructField(impressions,StringType,true),StructField(quantity,StringType,true),StructField(total_price,StringType,true)))

In [4]:


json_df = spark.readStream.option("maxFilesPerTrigger",5).schema(sc).format('json').option('header','true').load('/home/ubuntu/Spark/Spark_Work/python_file/data/json_files')


In [5]:

#json_df.printSchema()

json_df.isStreaming


True

In [6]:
df = json_df


df = df.withColumn('clicks',df.analytics.clicks)

df = df.withColumn('impressions',df.analytics.impressions)

df = df.withColumn('quantity',df.sales.quantity)
df = df.withColumn('total_price',df.sales.total_price)

df = df.drop("analytics","sales")

#f_df = df.select('datetime','clicks','impressions','quantity','total_price')

#f_df.schema


In [8]:



final_df = df

final_df = final_df.writeStream.outputMode("append").format("csv").option("path", "/home/ubuntu/Spark/Spark_Work/python_file/data/csv_files/").option("checkpointLocation", "/home/ubuntu/Spark/Spark_Work/python_file/data/checkpoint").option("header", "true").start()



22/08/10 12:33:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


22/08/10 23:39:20 ERROR MicroBatchExecution: Query [id = 94d71546-fe6c-4ee2-a6ff-6fe64613b280, runId = d84d55b3-785c-4976-afdf-dab48ced1962] terminated with error
java.io.FileNotFoundException: File file:/home/ubuntu/Spark/Spark_Work/python_file/data/checkpoint/sources/0 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.DelegateToFileSystem.getFileStatus(DelegateToFileSystem.java:128)
	at org.apache.hadoop.fs.DelegateToFileSystem.createInternal(DelegateToFileSystem.java:93)
	at org.apache.hadoop.fs.ChecksumFs$ChecksumFSOutputSummer.<init>(ChecksumFs.java:353)
	at org.apache.hadoop.fs.ChecksumFs.createInternal(ChecksumFs.java:400)
	at org.apache.hadoop.fs.AbstractFileSystem.create(AbstractFileSystem.java:626)
	at org.

In [8]:


final_df = df

final_df = final_df.writeStream.outputMode("append").format("csv").option("path", "/home/ubuntu/Spark/Spark_Work/python_file/data/csv_files/").option("checkpointLocation", "/home/ubuntu/Spark/Spark_Work/python_file/data/checkpoint").option("header", "true").start()



22/08/10 10:49:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
#final_df.isStreaming

df.printSchema()

True

In [1]:
spark.stop()

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


NameError: name 'spark' is not defined